# Agilepy quickstart notebook

This notebook will show you how to use the Agilepy library.

## The output directory must deleted first

The path to the output directory is defined in the configuration file (more on that later).

In [ ]:
import os
from pathlib import Path
from shutil import rmtree

In [ ]:
outDir = Path(os.environ["AGILE"]).joinpath( Path("agilepy-test-data/unittesting-output/jupyter/") )

if outDir.exists() and outDir.is_dir():
    print("Removing",outDir)
    rmtree(outDir)

In [ ]:
def readFile(filepath):
    with open(filepath, "r") as f:
        return f.readlines()

## Using the API class

We need to import the class in our scope

In [ ]:
from agilepy.api.AGAnalysis import AGAnalysis

The constructor need two files:
* a configuration file (yaml)
* a file containing a list of sources (xml format, or AGILE txt format)

For this tutorial, we already wrote them. If you want to look at all the possible configuration options, check the [Agilepy's documentation](https://agilepy.readthedocs.io/en/latest/manual/configuration_file.html).

Run the next cell to read the minimal configuration file we wrote:

In [ ]:
with open("./config_file.yaml") as f:
    print(f.read())

Run the next cell to read the xml file containing the descriptions of two sources:

In [ ]:
with open("./sources.xml") as f:
    print(f.read())

## Calling the constructor

You need to instantiate the class if you want to use it. 

The [constructor](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis) will setup a console logger which verbosity is defined by the *'verboselvl'* configuration option. A file logger is activated as well and its verbosity is always set to the maximum value.

The user-defined configuration options will override the default ones. A validation step is then performed: in case of wrong/malformed configuration options an Exception will be raised.  

All the sources present in *sources.xml* are loaded in-memory inside an object of class *SourcesLibrary* which is a member of the *AGAnalysis* class. 

In [ ]:
ag = AGAnalysis("./config_file.yaml", "./sources.xml")

## Managing the sources 

### Loading the sources into the SourcesLibrary

You can load other sources after the AGAnalysis object initialization, with the [loadSource(sourcesFilepath)](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.loadSource) method. 

This method let you to specify a rangeDist argument to filter out the sources which distance from (glon, glat) is not in the rangeDist interval. 

In [ ]:
ag.loadSources("./sources.xml", rangeDist = (0, 20))

### Getting the available catalogs
You can also get the available AGILE catalogs with the [getSupportedCatalogs()](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.getSupportedCatalogs) method.

In [ ]:
catalogs = ag.getSupportedCatalogs()
catalogs

In [ ]:
%%bash
head -n 3 /opt/anaconda3/envs/agilepy/agiletools/catalogs/2AGL.multi

### Converting a catalog from .txt format to .xml format 
If you want to work with the more convenient xml format, you can convert the source file from the txt format to the xml format, using the [convertCatalogToXml()](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.convertCatalogToXml) method.

In [ ]:
ag.convertCatalogToXml(catalogs[0])

In [ ]:
%%bash
head -n 18 /opt/anaconda3/envs/agilepy/agiletools/catalogs/2AGL.xml

## Generating maps

You can use the [generateMaps()](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.generateMaps) method to produce one or more sky maps of 4 different types (counts, exposure, gas and int) using the AGILE data. The method's behaviour is controlled by several configuration options. Check the [*'map'* section of the documentation](https://agilepy.readthedocs.io/en/latest/manual/configuration_file.html#maps) for more.

The method will return a maplist file, which format is [described here](https://agilepy.readthedocs.io/en/latest/manual/products.html#maplist-file). 

In [ ]:
mapfilepath = ag.generateMaps()

Let's print the content of the mapfile:

In [ ]:
with open(mapfilepath) as f:
    print(f.read())

### Parsing the mapfile

In [ ]:
maps = ag.getSkyMaps()

In [ ]:
print("Number of rows:",len(maps))

In [ ]:
ctsMap = maps[0][0]
ctsMap

In [ ]:
expMap = maps[0][1]
expMap

In [ ]:
gasMap = maps[0][2]
gasMap

## Starting an maximum likelyhood estimation analysis

Again, the behaviour of this analysis can be configured by sevaral configuration options. Check the [*'mle'* section of the documentation](https://agilepy.readthedocs.io/en/latest/manual/configuration_file.html#mle) for more.

The [mle()](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.mle) performs a maximum likelyhood estimation analysis and it produces one or more files with the '*.source*' extension. The '*.source*' files have a fixed format described [here](https://agilepy.readthedocs.io/en/latest/manual/products.html#source-file).

The mle analysis output is also saved inside the corresponding sources in the *SourcesLibrary* object. 

In [ ]:
source_files = ag.mle()

In [ ]:
for sf in source_files:
    print(sf)

## Interacting with the SourcesLibrary

You can interact with the SourcesLibrary throught the *AGAnalysis* class that provides you three useful methods.

### Get all the sources
You can obtain a list of all the sources with [getSources()](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.getSources).

In [ ]:
sources = ag.getSources()

In [ ]:
len(sources)

In [ ]:
for source in sources:
    print(source)

### Selecting sources

You can perform a query with [selectSources(selection)](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.selectSources).

The sources can be selected with the '*selection*' argument, supporting either lambda functions and boolean expression strings.

The selection criteria can be expressed using the following Source class’s parameters:

* name: the unique code identifying the source.
* dist: the distance of the source from the center of the maps.
* flux: the flux value.
* sqrtTS: the radix square of the ts.

The *sqrtTS* selection parameter can be used only after a call to mle().

#### Lamba selection

In [ ]:
sources = ag.selectSources(lambda name, dist, flux : name == "2AGLJ2021+4029" and dist > 2.5 and flux > 0)

In [ ]:
for source in sources:
    print(source)

#### Boolean string selection

In [ ]:
sources = ag.selectSources('name == "2AGLJ2021+4029" AND dist > 0 AND flux > 0')

In [ ]:
for source in sources:
    print(source)

### Free or Fix sources parameters

You can let some of the source parameters free to vary: in order to decide which parameters are free to vary and which parameters are fixed you can use the [freeSources(selection, parameterName, free)](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.freeSources) method.

The sources can be selected with the '*selection*' argument.

The '*parameterName*' argument is the name of the parameter you want to fix or let free to vary.

The '*free*' argument is a boolean:
* True: the parameter will be free to vary
* False: the parameter will be fixed.

The method returns the list of sources affected by the update.

In [ ]:
sources = ag.freeSources('name == "2AGLJ2021+4029" AND dist > 0 AND flux > 0', "flux", False)

In [ ]:
print(sources[0])

### Deleting sources

You can delete sources with [deleteSources(selection)](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.deleteSources).

The method returns the list of deleted sources.

In [ ]:
deletedSources = ag.deleteSources('flux > 0')

In [ ]:
for s in deletedSources:
    print(s)

We can check that the sources are deleted.

In [ ]:
sources = ag.getSources()

In [ ]:
len(sources)

### Adding a source
You can add a new source with [addSource(sourceName, sourceDict)](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.addSource).

You have to pass a dictionary containing the source's data.

In [ ]:
newSourceDict = {
    "glon" : 6.16978,
    "glat": -0.0676943,
    "spectrumType" : "LogParabola",
    "flux": 35.79e-08,
    "curvature": 0.682363
}
if ag.addSource("2AGLJ1801-2334", newSourceDict):
    print("Source loaded")

In [ ]:
if not ag.addSource("2AGLJ1801-2334", newSourceDict):
    print("Source NOT loaded, it is already present in the Sources Library")

## Updating the configuration options

You can update the configuration options using the [setOptions(**kwargs)](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.setOptions) method.

In [ ]:
ag.setOptions(binsize=0.6, mapsize=60, energybins=[[100, 1000], [1000, 3000]])

You can print on the console the configuration options values with the printOptions(section) method.

The section argument is optional but you can use to print a subset of the options.

In [ ]:
ag.printOptions("maps")

## Displaying the sky maps

You can display the sky maps using the [displaySkyMap()]() method.

In [ ]:
%matplotlib notebook

In [ ]:
ag.displaySkyMap(ctsMap, smooth=False)

You can also smooth the image

In [ ]:
ag.displaySkyMap(ctsMap, smooth=True, sigma=5)

Finally you can save it on disk

In [ ]:
outputfile = ag.displaySkyMap(expMap, smooth=True, sigma=5, saveImage=True)
print(outputfile,"saved.")